<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2024/blob/main/Unidad_2/01_Complejidad_en_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import time
import networkx as nx

import matplotlib.pyplot as plt

En esta sesión veremos algunos ejemplos de algoritmos (unos nuevos, y otros que ya hemos visto) y analizaremos sus complejidades, ya sea en tiempo, memoria o ambas.

# Preliminares

## Representación de enteros

En la gran mayoría de lenguajes de programación, los enteros son representados con un número fijo de bits. Esto significa que hay un límite superior e inferior a las cantidades que podemos expresar. Por ejemplo, para el tipo `int32` (enteros con signo de 32 bits), el rango de números válidos es:

$$
[-2^{31}, 2^{31}-1] = [-2\ 147\ 483\ 648, 2\ 147\ 483\ 647]
$$

Es importante notar que, bajo este esquema, todos los números ocupan el mismo espacio en memoria. Entonces, si queremos representar un 5 en `int32`, en realidad estamos guardando:

$$
5 = \underbrace{00000000000000000000000000000101}_\text{32 bits}
$$

---

Por alguna razón, los creadores de Python decidieron romper esta convención, y representar los enteros con un número *arbitrario* de bits. Esto significa que podemos expresar números tan grandes o tan pequeños como queramos, y Python asignará dinámicamente el número de bits necesario. A continuación, vemos algunos ejemplos de esto, utilizando el método `bit_length()` para obtener el número de bits:

In [2]:
enteros = [7, 15, 32, 1023, int(1e20)]
for i in enteros:
    print(f"Número de bits de {i}: {i.bit_length()}")

Si bien esto sirve para ahorrar memoria para enteros menores a 32 bits, lo logra sacrificando las **operaciones en tiempo constante**. Como en `int32` (e `int16`, `int8`, etc.), todos los números tienen el mismo número de bits, las operaciones algebraicas tardan en esencia el mismo tiempo, por lo cual puede decirse que son $O(1)$ (**Nota:** Este tiempo depende del número de bits: por ejemplo, las operaciones en `int32` tardan más que en `int16`).

Sin embargo, como en Python el número de bits es variable, las operaciones ya no son de tiempo constante. Para números pequeños, Python utiliza el algoritmo de multiplicación que aprendemos en primaria, con una complejidad $O(n^2)$. Por otro lado, para números muy grandes, emplea el algoritmo de Karatsuba, con complejidad $\approx O\left(n^{1.58}\right)$.

Para ilustrar esto, definimos la siguiente función que calcula el factorial de `n`:

In [1]:
def my_fac(n):
    s = 1
    for i in range(1, int(n)+1):
        s *= i
    return s

Claramente, su complejidad es $O(n)$, por lo tanto, esperamos que su gráfica de tiempo vs. $n$ sea una línea recta. Sin embargo, ejecutándola para distintos valores de $n$ y midiendo su tiempo:

In [4]:
times = []
ns = np.logspace(1, 5, 15) # Nos regresa una sucesión de 15 números espaciados uniformemente en [10^1, 10^5]
#print(ns)
for n in ns:
    res = %timeit -o -q my_fac(n)
    times.append(res.average)

Graficando:

In [15]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(ns, times, label="Esperado")
ax.set_xlabel("n")
ax.set_ylabel("Tiempo (s)")

Dado que en las últimas partes, estamos obteniendo $100000!$ (que requiere de un total de 1516688 bits para poder representarse por completo), estas complejidades ya no son triviales.

---

En caso de que la precisión no sea tan importante, podemos solucionar este problema usando `float` en vez de `int`. Como `float` tiene un tamaño fijo de 64 bits, la multiplicación siempre toma un tiempo constante:

In [16]:
def my_fac_2(n):
    s = 1.0
    for i in range(1,int(n)+1):
        s *= i
    return s

In [18]:
times = []
ns = np.logspace(1, 5, 15)
for n in ns:
    res = %timeit -o -q my_fac_2(n)
    times.append(res.average)

In [18]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(ns, times)
ax.set_xlabel("n")
ax.set_ylabel("Tiempo (s)")

## Pila de llamada

Python (y la mayoría de los lenguajes de programación) manejan lo que se conoce como una *pila de llamada* (en inglés *call stack*). Esta es  una estructura de datos de tipo LIFO (Last In First Out), en la cual se van colocando referencias a las funciones que vamos ejecutando. Cuando hacemos una llamada a una función, Python mete datos (variables, parámetros, punteros, etc.) a la pila; cuando la función termina de ejecutarse, la saca de la pila.

Ahora, supongamos que ejecutamos a la función `foo`. Después de iniciar, la pila se verá de la siguiente manera:

```
---
foo
---
```

Luego, si `foo` llama a la función `bar`, Python insertará la referencia correspondiente en la pila:

```
---
bar
---
foo
---
```

Cuando `bar` se acabe de ejecutar, saldrá de la pila:

```
---
foo
---
```

Y finalmente, `foo` termina, y sale de la pila:

```
---
---
```

---

¿Qué pasa cuando tenemos funciones *recursivas*? En este caso, Python insertará una referencia por cada una de las llamadas que hagamos. Por ejemplo, si tenemos la siguiente función para calcular el factorial de manera recursiva:

In [21]:
def fac(n):
    if n == 1:
        return 1
    return n * facc(n-1)

En total hará $n$ llamadas recursivas, por lo cual, al llegar a la última llamada (cuando $n=1$), la pila se verá de la siguiente manera:

```
---      --
fac<1>     \
---         \
fac<2>       \
---           \
fac<3>         n veces
---           /
...          /
---         /
fac<n>     /
---      --
```

Posteriormente, la función de hasta arriba ($n=1$) terminará y saldrá de la pila, seguida por la de abajo de ella, y así sucesivamente, hasta que se vacíe.

La complejidad de este algoritmo es:

* Tiempo: $O(n)$, ya que hacemos $n$ llamadas recursivas.
* Espacio: Como en la última llamada, la pila de llamadas tiene una altura $n$, la complejidad en espacio es $O(n)$, ya que la pila ocupa memoria física.

**Es importante tomar en cuenta la pila de llamadas cuando se analisa la complejidad en espacio.**

# Ejemplos

## Ejemplo 1

En la práctica de inducción, teníamos el siguiente problema:

Encuentra el valor de: 

$$\Big\lfloor \frac{1}{2}\Big \rfloor + \Big\lfloor \frac{2}{2}\Big \rfloor + \cdots + \Big\lfloor \frac{n}{2}\Big \rfloor$$

Este problema se puede resolver de forma sencilla utilizando un bucle `for`:

In [86]:
def floor_sum(n):
    s = 0
    for i in range(n+1):    # Se ejecuta n veces
        s += i//2           # Esta operación toma tiempo constante
    return s

* En tiempo, es $O(n)$, pues estamos haciendo $n$ operaciones, una en cada ciclo. 
* En memoria, es constante, pues lo único que tenemos que ir recordando es el valor de la suma en cada momento, es decir, es $O(1)$.

Nosotros habíamos encontrado una expresión cerrada para esta suma, con la cual podemos escribir otra función:

In [87]:
def floor_sum_2(n):
    return (n//2) * ((n+1)//2)

En este caso, la complejidad tanto en tiempo como en espacio es $O(1)$, ya que solo tenemos que realizar una operación sin importar el tamaño de la entrada.

## Ejemplo 2

Pasemos a un ejemplo más interesante. Supongamos que tenemos que diseñar un algoritmo tal que dado un entero positivo $n$ y un número real $x$, encuentre el valor de $x^n$, pero utilizando únicamente operaciones básicas (suma, resta, multiplicación y división).

Un primer algoritmo para hacer esto sería lo siguiente:

In [92]:
def potencia(x, n):
    p = x
    for i in range(2, n+1):
        p *= x
    return p

potencia(2,10)

* En tiempo, nos toma $O(n)$, pues estamos iterando $n$ veces y en cada una hacemos una cantidad constante de operaciones.
* En memoria, es $O(1)$, pues sólo actualizamos el valor de $p$ en cada iteración. 

---

¿Cómo podemos optimizarlo? 

Notemos que si $n = 2k$, entonces: 

$$x^n = x^{2k}= (x^k)^2=x^kx^k$$ 

Por otro lado, si $n = 2k+1$, entonces: 

$$x^n = (x^k)^2 \cdot x$$ 

Usemos esto para obtener un algoritmo que sea mejor en cuanto a tiempo de ejecución:

In [94]:
def potencia_2(x, n):
    if n == 1:                  # Comparaciones toman tiempo constante
        return x
    
    p = potencia_2(x, n//2)     # ¿Cuántas veces se hace la llamada recursica? 
    
    if n%2 == 0:                # Comparaciones toman tiempo constante
        return p*p              # Multiplicaciones toman tiempo constante
    else:
        return x*p*p            # Multiplicaciones toman tiempo constante

potencia_2(2, 10)

La complejidad de este algoritmo es:

* **Tiempo:** En cada llamada recursiva, dividimos el exponente entre 2. Por lo tanto, tenemos en promedio un total de $\log_2(n)$ llamadas, en cada una de las cuales hacemos operaciones de tiempo constante. Por lo tanto, la complejidad en tiempo es $O(\log_2(n))$.
* **Espacio:** De nuevo, hacemos un total de $\log_2(n)$ llamadas recursivas. Por lo tanto, en su punto más alto, la pila de llamadas tendrá  este tamaño, de lo cual se sigue que la complejidad en espacio es $O(\log_2(n))$.

## Ejemplo 3

Digamos que queremos calcular el $n$-ésimo valor de la sucesión de Fibonacci, i.e., $F_n$.

Una primera opción sería haciéndolo de manera recursiva:

In [19]:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n-1) + fib(n-2)

fib(8)

La complejidad de este algoritmo es:

* **Tiempo:** Como para cada $n > 1$, requerimos llamar a la función `fib` dos veces, se concluye que la complejidad en tiempo es $O(2^n)$.
* **Espacio:** La pila de recursión tiene como altura máxima $n$; por lo tanto, la complejidad en espacio es $O(n)$.

---

¿Qué pasa si guardamos los valores de los números de Fibonacci que vamos calculando? Es decir, guardamos el valor de $F_k$ la primera vez que llamemos a $fib(k)$.

In [53]:
def fib2(k, Fs=None):
    if Fs is None:
        Fs = [None] * (k+1)

    if k == 0:
        return 0
    elif k == 1:
        return 1
    
    if Fs[k] is not None:           # Si ya calculamos el valor fib(k), entonces no necesitamos calcularlo de nuevo
        return Fs[k]
    else:
        Fs[k] = fib2(k-1, Fs=Fs) + fib2(k-2, Fs=Fs) 
        return Fs[k]

print(fib2(8))

t = %timeit -o -q fib2(2500)
t

La complejidad de este algoritmo es:

* **Tiempo:** La complejidad en tiempo mejora considerablemente, ya que `fib(k)` solo llama a `fib(k-1)` y `fib(k-2)` una vez, y todas las demás veces simplemente se accesa el valor guardado. Dado que se tienen que hacer $n$ llamadas como mínimo, se concluye que la complejidad en tiempo es $O(n)$.
* **Espacio:** La altura máxima de la pila de llamadas no cambia. Adicionalmente, estamos guardando un arreglo de tamaño $n+1$ en la memoria, por lo cual la complejidad es $O(n) + O(n) = O(n)$.

A esta técnica de guardar los resultados de una función para utilizarlos después se le conoce como **memoización**, y la veremos con más detalle en libretas siguientes.


Otra técnica de optimización en funciones recursivas es la llamada recursión de cola (del inglés, tail recursion). En este tipo de recursión, lo que se hace es realizar la recursión como el último cómputo de la función recursiva.

In [55]:
def fib3(n, fib1 = 0, fib2 = 1):
    if n == 0:
        return fib1
    return fib3(n-1,  fib2, fib1+fib2)

t = %timeit -o -q fib3(2500)
t

La complejidad de este algoritmo es:

* **Tiempo:** Dado que se tienen que hacer $n$ llamadas recursivas, se concluye que la complejidad en tiempo es $O(n)$.
* **Espacio:** La altura de la pila siempre es 1, y no se hace uso de ninguna estructura de datos, por lo que la complejidad en espacio es $O(1)$.

A esta técnica de guardar los resultados de una función para utilizarlos después se le conoce como **memoización**, y la veremos con más detalle en libretas siguientes.

Estas funciones nos permiten encontrar el valor de $F_n$ de manera recursiva. Alternativamente, podemos escribir una que lo haga de manera iterativa:

In [54]:
def fib4(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    
    count = 1
    k1 = 0
    k2 = 1
    while count != n:
        k1, k2 = k2, k1 + k2
        count += 1
    return k2

fib3(8)

t = %timeit -o -q fib4(2500)
t

* **Tiempo:** $O(n)$, ya que hacemos $n$ iteraciones del bucle `while`.
* **Espacio:** $O(1)$, ya que las únicas variables que guardamos son `count, k1, k2`

Esto nos muestra algo que también es bastante común, hacer un algoritmo de forma iterativa suele hacerlo más eficiente, ya sea en espacio, en tiempo o ambas. Sin embargo, muchas veces es muy complicado pasar de algo recursivo a algo iterativo.

## Ejemplo 4

Analicemos las complejidades de el algoritmo de DFS, que vimos anteriormente.

In [120]:
n = 8
p = 1/3

G = nx.gnp_random_graph(n, p)
print(G.edges, '\n')
nx.draw(G, with_labels=True, font_weight='bold')

In [121]:
def dfs(g, v, visited=None):
    if visited is None:
        visited = [False] * len(g.nodes)
        
    visited[v] = True
    print(v) # Aquí puede ir cualquier otra cosa
    for u in g.adj[v]:
        if not visited[u]:
            dfs(g, u, visited)
            
dfs(G, 0)

Antes de iniciar el análisis, es importante entender cómo se representan las gráficas en las computadoras. Por ejemplo, tomemos la siguiente gráfica de ejemplo:

In [32]:
g = nx.Graph()
g.add_nodes_from([0, 1, 2, 3, 4])
g.add_edges_from([(0, 1), (0, 2), (0, 4), (1, 3), (3, 4)])

fig, ax = plt.subplots(figsize=(6, 4))
nx.draw(g, with_labels=True, font_weight="bold", font_color="white", ax=ax)

**Matriz de adyacencia**

La gráfica se representa como una matriz de $n\times n$. La entrada $(i, j)$ es 1 si existe una arista de $i$ a $j$, y 0 en otro caso. En caso de una gráfica no dirigida, la matriz es simétrica (i.e., tanto $(i, j)$ como $(j, i)$ es 1 si existe una arista).

Para nuestra gráfica de ejemplo, la matriz de adyacencia es:

$$
\begin{pmatrix}
0 & 1 & 1 & 0 & 1 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 \\
1 & 0 & 0 & 1 & 0 \\
\end{pmatrix}
$$

**Lista de adyacencia**

En este caso, la gráfica se representa como una lista de longitud $n$. La $i$-ésima entrada de la lista contiene todos los vértices a los cuales es adyacente el vértice $i$.

Entonces, para nuestra gráfica de ejemplo, la lista de adyacencia es:

```
0: 1, 2, 4
1: 0, 3
2: 0
3: 1, 4
4: 0, 3
```

---

¿Cómo influye esto en la complejidad del algoritmo?

Analicemos los casos por separado. Es importante recordar que estamos estudiando el algoritmo, por lo cual **no** debemos de considerar el espacio ocupado por la representación de la gráfica.

### Matriz de adyacencia

* **Tiempo:** Para obtener todos los vecinos de un nodo, tenemos que revisar cada entrada de la fila correspondiente de la matriz de adyacencia, lo cual toma tiempo $O(n)$. Como tenemos que hacer esto para todos los nodos, concluimos que la complejidad en tiempo es $O(n^2)$.
* **Espacio:** En el peor de los casos, haremos $n$ llamadas recursivas anidadas. Por lo tanto, la altura máxima de la pila de llamadas es $n$, y la complejidad en espacio es $O(n)$.

### Lista de adyacencia

* **Tiempo:** Para obtener todos los vecinos de un nodo, tenemos que accesar la correspondiente entrada en la lista de adyacencia, y revisar todos los nodos listados. Al finalizar el algoritmo, habremos accesado las $n$ entradas, y revisado los $m$ nodos listados. Por lo tanto, la complejidad en tiempo es $O(n+m)$.

Es importante notar que, para una gráfica no-dirigida, accesaremos cada arista dos veces; por lo tanto, la complejidad es $O(n+2m)\sim O(n+m)$.

* **Espacio:** Por el mismo argumento que en el caso anterior, tenemos que la complejidad en espacio es $O(n)$.

# Ejercicios

## Ejercicio 1

Como notamos en clase, la representación de matriz de adyacencia y lista de adyacencia es redundante para gráficas no-dirigidas. En el caso de la matriz de adyacencia, la siguiente matriz es equivalente a la de nuestro ejemplo:

$$
\begin{pmatrix}
0 & & & & \\
1 & 0 & & & \\
1 & 0 & 0 & & \\
0 & 1 & 0 & 0 & \\
1 & 0 & 0 & 1 & 0 \\
\end{pmatrix}
$$

Análogamente, nuestra lista de adyacencia puede reducirse a la siguiente forma:

```
0: 1, 2, 4
1: 3
2: 
3: 4
4: 
```

---

Asumiendo que tenemos estas representaciones para una gráfica en general, explica cómo cambiaría (si es que lo hace) la complejidad en tiempo y espacio del algoritmo DFS.

## Ejercicio 2

Podemos definir el máximo común divisor de $a$ y $b$ de manera recursiva de la siguiente manera:

$$
\text{mcd}(a, b) = 
\begin{cases}
a, & \text{si}\ b=0 \\
\text{mcd}(b, a\mod b), & \text{en otro caso} 
\end{cases}
$$

Escribe una función que utilice esta definición para calcular el MCD de manera recursiva. Posteriormente, escribe otra que lo haga de manera iterativa. Explica la complejidad en tiempo y espacio de ambas.

## Ejercicio 3

Escribe una función recursiva que determine si un *string* dado es un palíndromo o no. Luego, haz lo mismo de manera iterativa. Analiza la complejidad en tiempo y espacio de ambas.